In [2]:
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../data')


from utils import (init_results, 
                            get_dataset,
                            perform_experiment_simple,
                            SEED,
                            PRECOMPUTED_DISTANCES_PATH,
                            experiment_risf_complex,
                            experiment_risf_mixed)

import numpy as np
from netrd.distance import PortraitDivergence, DegreeDivergence, NetLSD
from risf.distance_functions import (GraphDist, manhattan_projection, cosine_projection, BagOfWord_projection, NumericalReprDist,
                                    euclidean_projection, DTWDist, EuclideanDist, WassersteinDist, CosineDist, BagOfWordDist, Goodall3Dist, OFDist,
                                    EditDistanceSequencesOfSets, ManhattanDist, ChebyshevDist, dummy_projection, LinDist, chebyshev_projection)
from risf.distance import SelectiveDistance
from pathlib import Path
import pandas as pd

%load_ext autoreload
%autoreload 2

np.random.seed(SEED)

In [3]:
PRECOMPUTED_DISTANCES_PATH.mkdir(exist_ok=True)

DATASETS = {
    #"graph":  ["NCI1", "AIDS", "ENZYMES", "PROTEINS_full", "DD"],
    #"numerical": [ "14_glass", "20_letter", "25_musk", "2_annthyroid", "31_satimage-2", "38_thyroid", "40_vowels", "41_Waveform",
    #"42_WBC" ,"43_WDBC", "44_Wilt",],
    #"binary":  ["AID362red_train_allpossiblenominal", "apascal_entire_trainvsall", "Reuters-corn-100"],
    #"nominal" : ["cmc-nominal", "solar-flare_FvsAll-cleaned"],
    #"timeseries": ["Earthquakes", "ECGFiveDays", "MiddlePhalanxOutlineCorrect", "SonyAIBORobotSurface1",],
    #"nlp" : ["amazon", "imdb", "yelp"],
    #"cv": ["CIFAR10_0", "FashionMNIST_0", "SVHN_0"],
    "multiomics": ["breast", "ovarian", "rosmap"],
    #"seq_of_sets": ["items", "length", "order"]

}

DATA_DIRS = {
    "graph": Path("../data/graph"),
    "numerical" : Path("../data/numerical"),
    "binary" : Path("../data/categorical"),
    "nominal" : Path("../data/categorical"),
    "timeseries": Path("../data/timeseries"),
    "nlp": Path("../data/adBench/NLP_by_RoBERTa/"),
    "cv": Path("../data/adBench/CV_by_ResNet18/"),
    "multiomics": Path("../data/mixed"),
    "seq_of_sets": Path("../data/mixed"),

}

def get_distances_risf(X, data_type):
    if data_type == "graph":
        return np.array([GraphDist(PortraitDivergence), SelectiveDistance(BagOfWord_projection, 1, 1),
                    GraphDist(DegreeDivergence), GraphDist(NetLSD), NumericalReprDist()], dtype=object)
    elif data_type == "numerical":
        return np.array([SelectiveDistance(manhattan_projection, 1, 3), 
                        SelectiveDistance(euclidean_projection, 1, 3), 
                        SelectiveDistance(cosine_projection, 2, 3),
                        SelectiveDistance(chebyshev_projection, 2, 3)], dtype=object)
    elif data_type == "timeseries":
        return np.array([SelectiveDistance(dummy_projection, 1, 1), SelectiveDistance(chebyshev_projection, 2, 2),
                SelectiveDistance(cosine_projection, 2, 2), DTWDist()], dtype=object)
    elif data_type in ["binary", "nominal"]:
        return np.array([SelectiveDistance(dummy_projection,1,1), LinDist(X), Goodall3Dist(X), OFDist(X)], dtype=object)
    elif data_type in ["nlp", "cv"]:
        return np.array([SelectiveDistance(manhattan_projection, 1, 3),SelectiveDistance(euclidean_projection, 1, 3), 
                SelectiveDistance(chebyshev_projection, 5, 5), SelectiveDistance(cosine_projection, 2, 2)], dtype=object)
    elif data_type == "histogram":
        return np.array([SelectiveDistance(dummy_projection,1,1), WassersteinDist()], dtype=object)
    elif data_type == "multiomics":
        return np.array([SelectiveDistance(dummy_projection,1,1), CosineDist(), EuclideanDist()], dtype=object)
    elif data_type == "seq_of_sets":
        return np.array([EditDistanceSequencesOfSets(), SelectiveDistance(dummy_projection, 1, 1), ManhattanDist()], dtype=object)
    
def get_distances_sf(X,data_type):
    if data_type == "graph":
        return np.array([GraphDist(PortraitDivergence), BagOfWordDist(),
                    GraphDist(DegreeDivergence), GraphDist(NetLSD), NumericalReprDist()], dtype=object)
    elif data_type == "numerical":
        return np.array([ManhattanDist(), EuclideanDist(), ChebyshevDist(), CosineDist()], dtype=object)
    elif data_type in ["binary", "nominal"]:
        return np.array([LinDist(X), Goodall3Dist(X), OFDist(X)], dtype=object)
    elif data_type in ["nlp", "cv"]:
        return np.array([ManhattanDist(), EuclideanDist(), ChebyshevDist(), CosineDist()], dtype=object)
    elif data_type == "timeseries":
        return np.array([CosineDist(), ChebyshevDist(), ManhattanDist() , DTWDist(), EuclideanDist()], dtype=object)
    elif data_type == "histogram":
        return np.array([WassersteinDist()], dtype=object)
    elif data_type == "seq_of_sets":
        return np.array([ManhattanDist(), CosineDist(), EditDistanceSequencesOfSets()], dtype=object)
    elif data_type == "multiomics":
        return np.array([CosineDist(), EuclideanDist()], dtype=object)


In [5]:
classifiers =  ["RISF", "ISF", "LOF", "IForest", "ECOD", "HBOS"]
SELECTED_OBJ_RATIO = 0.5
EXP_DIR = Path("../results/experiments/")
EXP_DIR.mkdir(parents=True, exist_ok=True)

for data_type in DATASETS.keys():
    print(data_type)
    results_all = []
    for clf in classifiers:
        print(clf)
        for dataset_name in DATASETS[data_type]:
            clf_kwargs = {} if clf != "RISF" else {}

            data = get_dataset(data_type, DATA_DIRS[data_type], dataset_name, clf)
            data['type'] = data_type
            if clf == "RISF":
                if data_type in ["numerical", "nlp", "cv"]:
                    distances = get_distances_risf(data["X"], data_type)
                    aucs = perform_experiment_simple(clf, data, clf_kwargs=clf_kwargs, distances = distances, optimize_distances=True)
                elif data_type in ["timeseries", "graph", "binary", "nominal", "seq_of_sets"]:
                    distances = get_distances_risf(data["X"], data_type)
                    aucs = experiment_risf_complex(clf, data, distances, selected_obj_ratio=SELECTED_OBJ_RATIO, clf_kwargs=clf_kwargs, optimize_distances=True)
                else:
                    feature_types = data["features_types"]
                    distances = []
                    for i, feature_type in enumerate(feature_types):
                        distances.append(get_distances_risf(data["X"][i], feature_type))
                        
                    aucs = experiment_risf_mixed(data, distances, selected_obj_ratio=SELECTED_OBJ_RATIO, clf_kwargs=clf_kwargs, optimize_distances=True)
                    
            elif clf == "ISF":
                distances = get_distances_sf(data["X"], data_type)
                aucs = experiment_risf_complex(clf, data, distances, selected_obj_ratio=1, clf_kwargs=clf_kwargs, optimize_distances=True)

            else:
                aucs = perform_experiment_simple(clf, data, clf_kwargs=clf_kwargs, optimize_distances=True)

            results = init_results(clf, dataset_name, data_type, aucs, clf_kwargs)
            results_all.extend(results)
                    
    pd.DataFrame(results_all).to_csv( EXP_DIR / f"{data_type}.csv", index=False)


multiomics
RISF
ISF
LOF
IForest
ECOD
HBOS


In [8]:
import pickle 

best_distances = pickle.load(open("../best_distances/RISF_Earthquakes.pickle", "rb"))

In [9]:
from risf.distance_functions import GraphDist
from risf.distance import SelectiveDistance
def get_shortcut(dist):
    if isinstance(dist, SelectiveDistance):
        return dist.projection_func.__name__
    if isinstance(dist, GraphDist):
        return dist.distance.__class__.__name__
    return dist.__class__.__name__

for fold_id, distances in best_distances.items():
    for n in range(4):
        distances_types = [get_shortcut(d) for d in distances[n][1]]
        print(f"for fold {fold_id}, {n}-th score is {distances[n][0]} and best distances are {distances_types}")
    print()
    print()

#TODO - check why when using NetLSD od AIDS we have 0.98 then 0.01

for fold 0, 0-th score is 0.7121333333333334 and best distances are ['dummy_projection', 'chebyshev_projection']
for fold 0, 1-th score is 0.7067 and best distances are ['chebyshev_projection']
for fold 0, 2-th score is 0.7034666666666668 and best distances are ['dummy_projection', 'cosine_projection']
for fold 0, 3-th score is 0.6818 and best distances are ['dummy_projection']


for fold 1, 0-th score is 0.7380666666666666 and best distances are ['chebyshev_projection']
for fold 1, 1-th score is 0.7099666666666667 and best distances are ['chebyshev_projection', 'cosine_projection']
for fold 1, 2-th score is 0.6948333333333334 and best distances are ['dummy_projection', 'chebyshev_projection', 'cosine_projection']
for fold 1, 3-th score is 0.6904666666666666 and best distances are ['dummy_projection', 'chebyshev_projection']


for fold 2, 0-th score is 0.6731333333333334 and best distances are ['chebyshev_projection', 'cosine_projection']
for fold 2, 1-th score is 0.6439333333333334 an